In [1]:
import tensorflow as tf
import numpy
import os
import json, pickle
import pandas
from functools import partial, reduce
import importlib

import sys
sys.path.append('../libs')

import flacdb
import prepare_data
import initialize
import data_pipeline
import conv_model
import plot_batch

In [13]:
metadata = pandas.read_hdf('/scr-ssd/mimic/metadata.hdf')

In [14]:
len(set(metadata['subject_id']))

10100

In [3]:
metadata

hadm_id  subject_id                    time gender        dob  \
rec_id  seg_id                                                                  
3000003 1        173995        9950 2155-04-19 19:44:07.664      M 2108-02-01   
        2        173995        9950 2155-04-19 19:46:22.685      M 2108-02-01   
        3        173995        9950 2155-04-19 19:46:25.757      M 2108-02-01   
        4        173995        9950 2155-04-19 19:46:33.981      M 2108-02-01   
        5        173995        9950 2155-04-19 19:46:37.053      M 2108-02-01   
...                 ...         ...                     ...    ...        ...   
3999988 6            -1          -1                     NaT    NaN        NaT   
        7            -1          -1                     NaT    NaN        NaT   
        8            -1          -1                     NaT    NaN        NaT   
        9            -1          -1                     NaT    NaN        NaT   
        10           -1          -1                     NaT    NaN        NaT   

                age  weight  height  sig_len death_time   race  
rec_id  seg_id                                                  
3000003 1        47     NaN      74    16878        NaT  white  
        2        47     NaN      74      384        NaT  white  
        3        47     NaN      74     1028        NaT  white  
        4        47     NaN      74      384        NaT  white  
        5        47     NaN      74   438826        NaT  white  
...             ...     ...     ...      ...        ...    ...  
3999988 6        -1     NaN      -1     5250        NaT    NaN  
        7        -1     NaN      -1      250        NaT    NaN  
        8        -1     NaN      -1      500        NaT    NaN  
        9        -1     NaN      -1      250        NaT    NaN  
        10       -1     NaN      -1    24875        NaT    NaN  

[2273298 rows x 11 columns]

In [4]:
sigdata = pandas.read_hdf('/scr-ssd/mimic/sig_data.hdf')

In [5]:
sigdata

,rec_id,seg_id,sig_index,sig_name,baseline,adc_gain,adc_res,adc_zero,fmt,init_value,checksum,units
0,3000003,1,0,II,0,24.0,8,0,80,0,-742,mV
1,3000003,1,1,V,0,24.0,8,0,80,0,0,mV
2,3000003,2,0,II,0,45.0,8,0,80,-9,-1948,mV
3,3000003,2,1,V,0,24.0,8,0,80,0,0,mV
4,3000003,3,0,II,0,29.0,8,0,80,-5,-3441,mV
...,...,...,...,...,...,...,...,...,...,...,...,...
8847983,3999988,10,0,RESP,307,409.0,10,512,16,255,-13867,pm
8847984,3999988,10,1,PLETH,0,1023.0,10,512,16,511,-2859,NU
8847985,3999988,10,2,III,255,512.0,10,512,16,0,0,mV
8847986,3999988,10,3,V,255,512.0,10,512,16,0,0,mV


In [28]:
len(set(metadata['subject_id']))

10100

In [11]:
metadata['death_time'].notna().mean()

0.04671011015713734

In [12]:
ls /scr-ssd/mimic

metadata.csv          metadata_old.hdf     sig_data.hdf          waveforms/
metadata.hdf          partition.pkl        sig_data_matched.csv
metadata_matched.csv  sig_data_backup.hdf  sig_data_old.hdf
metadata_old_2.hdf    sig_data.csv         test_subject_ids.txt


In [7]:
from initialize import *

In [11]:
diagnosis_init

,code,25000,2720,2724,2761,2762,27651,27652,27800,2859,4019,...,431,5119,51881,5715,5849,5859,78552,79902,99592,other
subject_id,hadm_id,,,,,,,,,,,,,,,,,,,,,
2,163353,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,145834,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
4,185777,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
5,178980,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
6,107064,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99985,176670,False,False,True,False,False,False,False,False,True,False,...,False,False,True,False,False,False,True,False,True,True
99991,151118,True,False,False,False,False,True,False,False,False,False,...,False,False,False,False,True,False,False,False,True,True
99992,197084,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True


In [10]:
diagnosis_augmented

,code,25000,2720,2724,2761,2762,27651,27652,27800,2859,4019,...,gender_F,gender_M,race_asian,race_black,race_hispanic,race_white,age_at_least_75,height_at_least_70,weight_at_least_100,died
subject_id,hadm_id,,,,,,,,,,,,,,,,,,,,,
20,157681,1,-1,1,-1,-1,-1,-1,-1,-1,1,...,1,-1,-1,-1,-1,1,1,0,0,-1
79,181542,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,0,0,0,0,-1,1,0,-1
107,174162,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,1,-1,-1,0,-1,-1
123,195632,1,-1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,1,-1,-1,1,-1,-1,1,0,-1
124,172461,-1,1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,1,-1,-1,-1,1,-1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99873,143544,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,-1,1,-1,1,1,-1
99883,150755,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,-1,1,-1,-1,-1,-1
99894,127380,-1,-1,1,-1,1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,1,-1,0,-1,-1


In [38]:
matched_data = metadata.reset_index()
matched_data = matched_data[matched_data['subject_id'] > 0]
matched_data.drop_duplicates(diagnosis.index.names, inplace=True)
matched_data.set_index(diagnosis.index.names, inplace=True)
matched_data.sort_index(inplace=True)
matched_data

,,rec_id,seg_id,time,gender,dob,age,weight,height,sig_len,death_time,race,chunk_count
subject_id,hadm_id,,,,,,,,,,,,
20,157681,3544749,5,2183-04-28 17:48:34.810,F,2107-06-13,75,NaN,-1,7098177,NaT,white,16
79,181542,3842928,5,2175-09-26 01:31:04.128,M,2123-06-11,52,NaN,70,4694782,NaT,NaN,16
107,174162,3860035,29,2122-05-14 11:47:31.000,M,2052-04-02,70,84.000000,-1,1186875,NaT,hispanic,16
123,195632,3485814,7,2161-10-29 03:31:56.271,M,2105-08-31,56,NaN,73,2546506,NaT,hispanic,16
124,172461,3255538,3,2160-07-05 22:37:40.963,M,2090-11-19,69,NaN,-1,1339511,NaT,white,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99873,143544,3469554,4,2103-12-20 14:01:32.000,M,2058-05-08,45,103.400002,74,87250,NaT,white,1
99883,150755,3080175,3,2131-12-24 04:39:49.496,M,2058-02-04,73,61.599998,65,391375,NaT,white,5
99894,127380,3974697,3,2129-07-07 02:51:21.880,F,2069-07-03,60,69.400002,-1,75125,NaT,white,1


In [9]:
diagnosis

,code,25000,2720,2724,2761,2762,27651,27652,27800,2859,4019,...,gender_F,gender_M,race_asian,race_black,race_hispanic,race_white,age_at_least_75,height_at_least_70,weight_at_least_100,died
subject_id,hadm_id,,,,,,,,,,,,,,,,,,,,,
20,157681,1,-1,1,-1,-1,-1,-1,-1,-1,1,...,1,-1,-1,-1,-1,1,1,0,0,-1
79,181542,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,0,0,0,0,-1,1,0,-1
107,174162,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,1,-1,-1,0,-1,-1
123,195632,1,-1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,1,-1,-1,1,-1,-1,1,0,-1
124,172461,-1,1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,1,-1,-1,-1,1,-1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99873,143544,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,-1,1,-1,1,1,-1
99883,150755,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,-1,1,-1,-1,-1,-1
99894,127380,-1,-1,1,-1,1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,1,-1,0,-1,-1


In [47]:
ids = metadata.reset_index()['subject_id']
len(set(ids[ids>0]))

10067

In [42]:
S = S4 | S5
sorted(S)[:5]

[(3000003, 1), (3000003, 2), (3000003, 3), (3000003, 4), (3000003, 5)]

In [58]:
recs = [i[0] for i in S]

In [63]:
sub_recs = {(i, j) for i, j, k in metadata.index if i > 0}

In [66]:
recs = {i for i, j in S}

In [67]:
len({i for i, j in sub_recs if j in recs})

10067

In [18]:
import importlib

In [2]:
import initialize

In [15]:
importlib.reload(initialize)

<module 'initialize' from '../libs/initialize.py'>

In [45]:
load_path = '/scr1/mimic/initial_data/'
sig_data, metadata = initialize.load_initial_data()
H = initialize.load_hypes()
diagnosis_init = initialize.load_diagnosis(H['icd_codes'])
diagnosis_augmented = initialize.augment_diagnosis(diagnosis_init, metadata)
diagnosis = initialize.fix_diagnosis(diagnosis_augmented)
priors = (diagnosis == 1).sum() / (diagnosis != 0).sum()
priors

code
25000                  0.201369
2720                   0.112389
2724                   0.276717
2761                   0.085799
2762                   0.110354
27651                  0.024648
27652                  0.028288
27800                  0.044666
2859                   0.134753
4019                   0.465626
41071                  0.074673
41401                  0.300719
4160                   0.009669
4240                   0.065480
4241                   0.069275
42731                  0.302998
42732                  0.033991
4275                   0.037180
4280                   0.300514
42832                  0.032799
42833                  0.037945
431                    0.044108
5119                   0.063844
51881                  0.174434
5715                   0.032131
5849                   0.209651
5859                   0.087646
78552                  0.062356
79902                  0.034464
99592                  0.085592
gender_F               0.497984
gen

In [18]:
(diagnosis == 1).any(level=0, axis=0).sum()

25000                  1724
2720                    991
2724                   2397
2761                    758
2762                    977
27651                   221
27652                   256
27800                   394
2859                   1199
4019                   4090
41071                   674
41401                  2629
4160                     85
4240                    579
4241                    615
42731                  2616
42732                   301
4275                    336
4280                   2517
42832                   281
42833                   322
431                     396
5119                    562
51881                  1536
5715                    271
5849                   1800
5859                    756
78552                   547
79902                   305
99592                   750
gender_F               4328
gender_M               5739
race_asian              239
race_black              817
race_hispanic           343
race_white          

In [19]:
(diagnosis == 1).any(level=0, axis=0).sum()

25000                   900
2720                    555
2724                   1148
2761                    372
2762                    564
27651                    70
27652                    98
27800                   212
2859                    576
4019                   2106
41071                   360
41401                  1414
4160                     40
4240                    303
4241                    358
42731                  1419
42732                   154
4275                    233
4280                   1278
42832                    98
42833                   119
431                     218
5119                    342
51881                  1015
5715                    151
5849                    838
5859                    317
78552                   386
79902                    86
99592                   497
gender_F               2123
gender_M               2941
race_asian              113
race_black              335
race_hispanic           165
race_white          

In [17]:
diagnosis

,code,4019,41071,41401,4240,4241,42731,42732,4275,4280,42833,...,gender_F,gender_M,race_asian,race_black,race_hispanic,race_white,age_at_least_75,height_at_least_70,weight_at_least_100,died
subject_id,hadm_id,,,,,,,,,,,,,,,,,,,,,
20,157681,1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,1,1,0,0,-1
79,181542,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,0,0,0,0,-1,1,0,-1
107,174162,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,1,-1,-1,0,-1,-1
123,195632,1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,1,-1,-1,1,0,-1
124,172461,1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,-1,1,-1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99873,143544,-1,-1,1,1,-1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,1,-1,1,1,-1
99883,150755,-1,-1,1,-1,-1,1,-1,-1,1,-1,...,-1,1,-1,-1,-1,1,-1,-1,-1,-1
99894,127380,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,1,-1,0,-1,-1


In [20]:
diagnosis.columns

Index(['25000', '2720', '2724', '2761', '2762', '27651', '27652', '27800',
       '2859', '4019', '41071', '41401', '4160', '4240', '4241', '42731',
       '42732', '4275', '4280', '42832', '42833', '431', '5119', '51881',
       '5715', '5849', '5859', '78552', '79902', '99592', 'gender_F',
       'gender_M', 'race_asian', 'race_black', 'race_hispanic', 'race_white',
       'age_at_least_75', 'height_at_least_70', 'weight_at_least_100', 'died'],
      dtype='object', name='code')

In [42]:
root = '/scr1/mimic/'
clinic_file = lambda i: root + 'clinic/{}.csv'.format(i.upper())

In [44]:
date_cols = ['DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN']
patients = pandas.read_csv(clinic_file('patients'), parse_dates=date_cols)
patients.set_index('SUBJECT_ID', verify_integrity=True, inplace=True)
patients

,ROW_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
SUBJECT_ID,,,,,,,
249,234,F,2075-03-13,NaT,NaT,NaT,0
250,235,F,2164-12-27,2188-11-22,2188-11-22,NaT,1
251,236,M,2090-03-15,NaT,NaT,NaT,0
252,237,M,2078-03-06,NaT,NaT,NaT,0
253,238,F,2089-11-26,NaT,NaT,NaT,0
...,...,...,...,...,...,...,...
44089,31840,M,2026-05-25,NaT,NaT,NaT,0
44115,31841,F,2124-07-27,NaT,NaT,NaT,0
44123,31842,F,2049-11-26,2135-01-12,2135-01-12,NaT,1


In [58]:
hdrs_m = pickle.load(open('/scr1/mimic/headers_matched_.pkl', 'rb'))

In [61]:
len(hdrs_m)

914836

In [97]:
hdrs_m[0]

{'record_name': 'p000020-2183-04-28-17-47',
 'n_sig': 4,
 'fs': 125,
 'counter_freq': None,
 'base_counter': None,
 'sig_len': 9862593,
 'base_time': datetime.time(17, 47, 59, 486000),
 'base_date': datetime.date(2183, 4, 28),
 'comments': [],
 'sig_name': None,
 'layout': 'variable',
 'segments': None,
 'seg_name': ['3544749_layout',
  '~',
  '3544749_0001',
  '3544749_0002',
  '3544749_0003',
  '3544749_0004',
  '3544749_0005',
  '3544749_0006',
  '3544749_0007',
  '3544749_0008'],
 'seg_len': [0, 93, 3811, 124, 8, 380, 7098177, 15000, 30000, 2715000],
 'sig_segments': None,
 'n_seg': 10,
 'base_datetime': datetime.datetime(2183, 4, 28, 17, 47, 59, 486000)}

In [ ]:
sum('layouti['record_name'])

In [62]:
import glob

In [70]:
ls /scr1/mimic/headers/matched

DOI            local.css  p02/  p05/  p08/     RECORDS-numerics
HEADER.shtml   p00/       p03/  p06/  p09/     RECORDS-waveforms
HEADER.shtml~  p01/       p04/  p07/  RECORDS


In [19]:
def get_matched_rec_segs():
    hdrs = glob.glob('/scr1/mimic/headers/matched/p0*/*/*.hea')
    hdrs = [i.split('/')[-1].split('.')[0] for i in hdrs]
    rec_segs = [
        i.split('_') for i in hdrs 
        if 'layout' not in i and 'p' not in i
    ]
    rec_segs = {(int(i), int(j)) for i, j in rec_segs}
    return rec_segs

In [20]:
def get_matched_rec_segs_old():
    rec_segs = [i['record_name'] for i in hdrs_m]
    rec_segs = [
        i.split('_') for i in rec_segs 
        if 'layout' not in i and 'p' not in i
    ]
    rec_segs = {(int(i), int(j)) for i, j in rec_segs}
    return rec_segs

In [21]:
def get_general_rec_segs():
    hdrs = glob.glob('/scr1/mimic/headers/3*/*/*.hea')
    hdrs = [i.split('/')[-1].split('.')[0] for i in hdrs]
    rec_segs = [
        i.split('_') for i in hdrs 
        if 'layout' not in i and 'p' not in i
    ]
    rec_segs = {(int(i[0]), int(i[1])) for i in rec_segs if len(i) == 2}
    return rec_segs

In [22]:
def get_downloaded_rec_segs():
    rec_segs = os.listdir(flacdb.ROOT)
    to_tuple = lambda i: flacdb.rec_seg_to_tuple(i.split('.')[0])
    S1 = {to_tuple(i) for i in rec_segs if '_x.flac' in i}
    S2 = {to_tuple(i) for i in rec_segs if '_x.hea' in i}
    S3 = {to_tuple(i) for i in rec_segs if '_y.hea' in i}
    S4 = {to_tuple(i) for i in rec_segs if '_y.flac' in i}
    S5 = {to_tuple(i) for i in rec_segs if '.hea' in i and '_x' not in i and '_y' not in i}
    S = S1 & S2 & S3 & S4 & S5
    return S

In [23]:
def get_new_rec_segs():
    S = glob.glob('/scr1/mimic/new_data/*.flac')
    S = [i.split('/')[-1].split('.')[0].split('_') for i in S if '.flac' in i]
    S = {(int(i[0]), int(i[1])) for i in S if len(i) == 2}
    return S

In [28]:
import glob

In [29]:
S5 = get_new_rec_segs()

In [30]:
S3 = get_general_rec_segs()

In [31]:
S1 = get_matched_rec_segs()

In [32]:
S2 = get_matched_rec_segs_old()

NameError: name 'hdrs_m' is not defined

In [36]:
S4 = get_downloaded_rec_segs()

In [37]:
len(S6 & S4)

566217

In [34]:
len(S6 & S5)

632726

In [41]:
len(S6 & (S4 | S5))

1198943

In [141]:
len(S1 - (S4 | S5))

2

In [33]:
metadata = pandas.read_hdf('/scr-ssd/mimic/metadata.hdf')
S6 = metadata.reset_index()[['rec_id', 'seg_id']].values
S6 = {(i, j) for i, j in S6}

In [149]:
len(S1)

866338

In [40]:
len(S6 - S1)

1406960

In [39]:
len(S6)

2273298

In [38]:
len(S6 - (S4 | S5))

1074355

In [125]:
len(S2 - S3)

0

In [112]:
len(S1)

866338

In [113]:
len(S1 & S2)

866338

In [115]:
len(S2 - S1)

3934

In [155]:
{i for i, j in S6 - (S4 | S5)}

{3661824,
 3063812,
 3563527,
 3072013,
 3031061,
 3006488,
 3448857,
 3768355,
 3121187,
 3039268,
 3776550,
 3022887,
 3170352,
 3031096,
 3006521,
 3162170,
 3031106,
 3194953,
 3072073,
 3047501,
 3178573,
 3547219,
 3178586,
 3104866,
 3137639,
 3113067,
 3113072,
 3637363,
 3145849,
 3170427,
 3793028,
 3113109,
 3031190,
 3170460,
 3440803,
 3072168,
 3719345,
 3653809,
 3014838,
 3440822,
 3555519,
 3162308,
 3113166,
 3023058,
 3162323,
 3162326,
 3793117,
 3481823,
 3121390,
 3088623,
 3162354,
 3105015,
 3096825,
 3186953,
 3105035,
 3793170,
 3498259,
 3449113,
 3178777,
 3080475,
 3416347,
 3006759,
 3039532,
 3039541,
 3416384,
 3096901,
 3146058,
 3187021,
 3113296,
 3047774,
 3080544,
 3416416,
 3162466,
 3080556,
 3015021,
 3457390,
 3768697,
 3121544,
 3031433,
 3162504,
 3072395,
 3170716,
 3531176,
 3170729,
 3056043,
 3097006,
 3137968,
 3178929,
 3162546,
 3056051,
 3097020,
 3121603,
 3629516,
 3645903,
 3162580,
 3015124,
 3187167,
 3629546,
 3056107,
 3146219,


In [117]:
{i for i, j in S2 - S1}

{3783537}

In [105]:
len(rec_segs)

870272

In [ ]:
len()

In [60]:
hdrs_m[0]

{'record_name': 'p000020-2183-04-28-17-47',
 'n_sig': 4,
 'fs': 125,
 'counter_freq': None,
 'base_counter': None,
 'sig_len': 9862593,
 'base_time': datetime.time(17, 47, 59, 486000),
 'base_date': datetime.date(2183, 4, 28),
 'comments': [],
 'sig_name': None,
 'layout': 'variable',
 'segments': None,
 'seg_name': ['3544749_layout',
  '~',
  '3544749_0001',
  '3544749_0002',
  '3544749_0003',
  '3544749_0004',
  '3544749_0005',
  '3544749_0006',
  '3544749_0007',
  '3544749_0008'],
 'seg_len': [0, 93, 3811, 124, 8, 380, 7098177, 15000, 30000, 2715000],
 'sig_segments': None,
 'n_seg': 10,
 'base_datetime': datetime.datetime(2183, 4, 28, 17, 47, 59, 486000)}

In [ ]:
for i in 

In [57]:
len(set(patients.index))

46520

In [46]:
patients['EXPIRE_FLAG'].mean()

0.33875752364574374

In [51]:
ids = metadata.reset_index()['subject_id']
len(set(ids[ids>0]))

5064

In [55]:
len(set(diagnosis.reset_index()['subject_id']))

5064

In [56]:
metadata['gender'].value_counts()

M    47773
F    34912
Name: gender, dtype: int64